In [4]:
import sys  
sys.path.insert(0, '../')

from src.evaluation import *
import src.utils as utils
# import pandas as pd


normal_data = utils.load_dataset("/home/wallat/temporal-llms/data/ArchivalQA/splits/ArchivalQATime/reference_types/sample/absolute_time_reference.csv")
normal_predictions = utils.load_dataset("/home/wallat/temporal-llms/data/predictions/referencing_samples/alpaca-7b_ArchivalQA_abolute-time_sample_gpt_style_predictions.csv")

no_time_data = utils.load_dataset("/home/wallat/temporal-llms/data/ArchivalQA/splits/ArchivalQATime/reference_types/sample/no_time_reference.csv")
no_time_predictions = utils.load_dataset("/home/wallat/temporal-llms/alpaca-7b_ArchivalQA_no_time_sample_gpt_style_predictions.csv")

normal_data.head()



,id,Question,Answer,org_answer,answer_start,para_id,trans_que,trans_ans,source
0,train_386299,What hurricane hit Cape May in 1993?,Hurricane Emily,Hurricane Emily,252,1024450_7,0,0,rand
1,train_316690,What manned landing on the Moon ended in 1972?,Apollo,Apollo,149,229402_13,0,0,wiki
2,train_227492,How many drivers died in the crash in 1973?,two,two,130,1077994_6,0,0,wiki
3,train_218845,How many vehicles did Toyota sell in 1993?,"1,033,211","1,033,211",206,660140_6,0,0,rand
4,train_222155,What airline filed for Chapter 11 bankruptcy i...,Braniff Airways Inc.,Braniff Airways Inc.,9,211708_1,0,0,wiki


In [5]:
# Evaluate


def evaluate_and_get_scores(data, model_predictions):
    dataset = split_answers_with_multiple_options(data, "")

    references = convert_to_references(dataset)
    predictions = convert_to_predictions(model_predictions)
    questions = extract_questions(dataset)

    contains, scores = compute_contains_metric(references, predictions)

    return contains, scores


normal_contains, normal_scores = evaluate_and_get_scores(normal_data, normal_predictions)
normal_contains

17.466666666666665

In [8]:
normal_scores[:10]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
no_time_contains, no_time_scores = evaluate_and_get_scores(no_time_data, no_time_predictions)
no_time_contains

14.266666666666666

In [11]:
# Confusion matrix 
from sklearn.metrics import confusion_matrix
confusion_matrix(normal_scores, no_time_scores)

array([[2402,   74],
       [ 170,  354]])

In [12]:
tp = 0
fp = 0
tn = 0
fn = 0

for time_pred, no_time_pred in zip(normal_scores, no_time_scores):
    if time_pred == 1 and no_time_pred == 1:
        tp += 1
    elif time_pred == 1 and no_time_pred == 0:
        fn += 1
    elif time_pred == 0 and no_time_pred == 1:
        fp += 1
    elif time_pred == 0 and no_time_pred == 0:
        tn += 1

print("tp: ", tp)
print("fp: ", fp)
print("tn: ", tn)
print("fn: ", fn)


tp:  354
fp:  74
tn:  2402
fn:  170


In [13]:
print(len(normal_scores))

3000
